<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [0]:
import pandas as pd
import psycopg2
import getpass

Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'
icd9_code = '99592'

Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

Load Hadm_id of Patients suffering from severe sepsis

In [14]:
query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code LIKE '99592';
"""
admissions = pd.read_sql_query(sql=query,con=conn)

,icd9_code,hadm_id,row_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,row_id,subject_id,seq_num,row_id,short_title,long_title
0,99592,130570,23936,19592,2155-05-14 16:43:00,2155-05-20 17:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,...,NaT,SEPSIS,0,1,218786,19592,2,11404,Severe sepsis,Severe sepsis
1,99592,175793,4984,4109,2132-10-28 15:08:00,2132-11-06 20:20:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,...,2132-10-28 16:47:00,PNEUMONIA,0,1,46021,4109,8,11404,Severe sepsis,Severe sepsis
2,99592,170070,32881,26896,2129-03-22 19:45:00,2129-03-28 16:51:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,...,2129-03-22 21:04:00,SEPSIS,0,1,301301,26896,9,11404,Severe sepsis,Severe sepsis
3,99592,119848,49756,70924,2132-10-11 21:36:00,2132-11-04 19:08:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,...,2132-10-11 22:44:00,PANCREATITIS,0,1,521565,70924,12,11404,Severe sepsis,Severe sepsis
4,99592,144636,10386,8495,2185-05-20 11:16:00,2185-05-30 14:05:00,NaT,EMERGENCY,TRANSFER FROM SKILLED NUR,REHAB/DISTINCT PART HOSP,...,NaT,CHRONIC OBST PULM DISEASE;SEPSIS,0,1,95013,8495,10,11404,Severe sepsis,Severe sepsis


List Items for Consideration

In [0]:
d_items = ['Platelets','C Reactive Protein (CRP)','pH (Art)','HCO3','TCO2 (cap)']
d_labitems = ['Leukocytes','Urea Nitrogen','Creatinine','Glucose','Calcium, Total','Sodium','Potassium','Bilirubin','Albumin','Lactate','pCO2','pO2','Hemoglobin']

In [0]:
query = """
SELECT DISTINCT * FROM mimiciii.d_items WHERE label IN ('Platelets','C Reactive Protein (CRP)','pH (Art)','HCO3','TCO2 (cap)');
"""
d_items = pd.read_sql_query(sql=query,con=conn)
query = """
SELECT DISTINCT * FROM mimiciii.d_labitems WHERE label IN ('Leukocytes','Urea Nitrogen','Creatinine','Glucose','Calcium, Total','Sodium','Potassium','Bilirubin','Albumin','Lactate','pCO2','pO2','Hemoglobin');
"""
d_labitems = pd.read_sql_query(sql=query,con=conn)

In [27]:
d_labitems['itemid']

0     50809
1     50811
2     50813
3     50818
4     50821
5     50833
6     50862
7     50893
8     50912
9     50931
10    50971
11    50983
12    51006
13    51222
14    51464
15    51478
16    51486
Name: itemid, dtype: int64

Fetch records for each admission

In [26]:
d_items

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,731,812,HCO3,None,carevue,chartevents,ABG,None,None,None
1,747,828,Platelets,None,carevue,chartevents,Hematology,None,None,None
2,1630,4753,pH (Art),None,carevue,chartevents,ABG,None,None,None
3,2229,4199,TCO2 (cap),None,carevue,chartevents,Other ABGs,None,None,None
4,5340,30006,Platelets,None,carevue,inputevents_cv,None,None,None,None
5,13768,225170,Platelets,Platelets,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,None
6,14758,227444,C Reactive Protein (CRP),C Reactive Protein (CRP),metavision,chartevents,Labs,None,Numeric with tag,None


In [20]:
admissions

,icd9_code,hadm_id,row_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,row_id,subject_id,seq_num,row_id,short_title,long_title
0,99592,130570,23936,19592,2155-05-14 16:43:00,2155-05-20 17:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,...,NaT,SEPSIS,0,1,218786,19592,2,11404,Severe sepsis,Severe sepsis
1,99592,175793,4984,4109,2132-10-28 15:08:00,2132-11-06 20:20:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,...,2132-10-28 16:47:00,PNEUMONIA,0,1,46021,4109,8,11404,Severe sepsis,Severe sepsis
2,99592,170070,32881,26896,2129-03-22 19:45:00,2129-03-28 16:51:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,...,2129-03-22 21:04:00,SEPSIS,0,1,301301,26896,9,11404,Severe sepsis,Severe sepsis
3,99592,119848,49756,70924,2132-10-11 21:36:00,2132-11-04 19:08:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,...,2132-10-11 22:44:00,PANCREATITIS,0,1,521565,70924,12,11404,Severe sepsis,Severe sepsis
4,99592,144636,10386,8495,2185-05-20 11:16:00,2185-05-30 14:05:00,NaT,EMERGENCY,TRANSFER FROM SKILLED NUR,REHAB/DISTINCT PART HOSP,...,NaT,CHRONIC OBST PULM DISEASE;SEPSIS,0,1,95013,8495,10,11404,Severe sepsis,Severe sepsis
